In [16]:
#!/usr/bin/python
__author__ = "Xin Xin"
__email__ = "xin.xin@yale.edu"
__copyright__ = "Copyright 2022"
__license__ = "GPL"
### Usage: python hw1.py -i <input file> -s <score file>
### Example: python hw1.py -i input.txt -s blosum62.txt
### Note: Smith-Waterman Algorithm

### Available at: https://github.com/xinxin1219/SWalgorithm/

import argparse
import pandas as pd
import numpy as np

# overall run SW function
def runSWalgorithm(inFile, scoreFile, open_gap_score, extension_score):
    ### calculation
    ### write output
    # to read in files and define two seq 
    score_file = pd.read_csv(scoreFile,delim_whitespace=True)     
    inputdata = pd.read_table(inFile, header = None)
    seq1 = np.array(inputdata)[0][0]
    seq2 = np.array(inputdata)[1][0]

    # to create an empty score matrix with all 0 values
    score_matrix=np.zeros((len(seq1) + 1, len(seq2) + 1), np.int)
    
    # to start building the score matrix
    for j in range(1,len(seq1)+1):    
        for i in range(1,len(seq2)+1):

            # deletion
            max_gap_len=j     
            deletion_all=np.array([])
            for m in range(1, max_gap_len+1):
              deletion_all=np.append(deletion_all, score_matrix[j-1-(m-1),i]+open_gap_score+(m-1)*extension_score)
            deletion_score=score_matrix[j,i]+np.max(deletion_all)

            # insertion
            max_gap_len=i
            insertion_all=np.array([])
            for m in range(1, max_gap_len+1):
              insertion_all=np.append(insertion_all, score_matrix[j,i-1-(m-1)]+open_gap_score+(m-1)*extension_score)
            insertion_score=score_matrix[j,i]+np.max(insertion_all)

            # matched values: from diagnoal 
            match_mismatch = score_matrix[j - 1, i - 1] + score_file.loc[seq1[j-1],seq2[i-1]] 

            # to obtain the max value, if negative then set to 0 value
            maxvalue = max(match_mismatch,deletion_score,insertion_score,0)

            if maxvalue > 0:
                score_matrix[j,i] = maxvalue

    # to obtain the location for the score_matrix with the max value
    score_loc = np.where(score_matrix==np.max(score_matrix)) 
    return seq1, seq2, score_matrix, score_loc


# to obtain where the largest value from, diag or fromcol or from row
def from_where(matrix,i,j):
    diag = matrix[i-1,j-1]
    fromcol = matrix[i-1,j]
    fromrow = matrix[i,j-1]
    # 1 means a diag move, 0 means stop 
    if diag >= fromcol and diag >= fromrow:     
      return 1 if diag != 0 else 0    
    # 2 means move fromcol or end action.
    elif fromcol > diag and fromcol >= fromrow:     
      return 2 if fromcol != 0 else 0 
    # 3 means move fromrow or end action.     
    elif fromrow > diag and fromrow > fromcol:
      return 3 if fromrow != 0 else 0    
    else:
      raise ValueError('not valid')


# traceback
def traceback(matrix, max_loc, seq_1, seq_2):
    # to set four actions, empty aligned seq, location of max value
    END, DIAG, FROMCOL, FROMROW = range(4)
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = max_loc

    # to use another from_where function to obtain the matrix of locations
    move = from_where(matrix, i, j)
    m = 0
    # to start different actions
    while move != END:
        # from diag (match)
        if move == DIAG:
            aligned_seq1.append(seq_1[i[0] - 1])
            aligned_seq2.append(seq_2[j[0] - 1])
            i -= 1
            j -= 1
        # from col
        elif move == FROMCOL:
            aligned_seq1.append(seq_1[i[0] - 1])
            aligned_seq2.append('-')
            i -= 1
            m +=1
        # from row
        else:
            aligned_seq1.append('-')
            aligned_seq2.append(seq_2[j[0] - 1])
            j -= 1

        move = from_where(matrix, i, j)

    aligned_seq1.append(seq_1[i[0] - 1])
    aligned_seq2.append(seq_2[j[0] - 1])
    aligned_seq1=''.join(reversed(aligned_seq1))
    aligned_seq2=''.join(reversed(aligned_seq2))

    return aligned_seq1, aligned_seq2, m



# to print out the output matching result according to the output sample
def printout_matrix(seq_1,seq_2,matrix,alignedseq1,alignedseq2,num):
    print("-----------\n|Sequences|\n-----------")
    print('sequence1')
    print(seq_1)
    print('sequence2')
    print(seq_2)

    print("--------------\n|Score Matrix|\n--------------")
    show_score_matrix = pd.DataFrame(matrix)
    show_score_matrix.index = np.append('',list(map(str, seq_1)))
    show_score_matrix.columns = np.append('',list(map(str, seq_2)))
    show_score_matrix = show_score_matrix.T
    finalmax = np.max(matrix).astype("int64")
    print(show_score_matrix.to_csv(sep = '\t'),end='')

    print("----------------------\n|Best Local Alignment|\n----------------------")
    output1 = 'Alignment Score:' + str(finalmax)
    print(output1)

    print("Alignment Results:")
    alignment_string = []
    for base1, base2 in zip(alignedseq1, alignedseq2):
      if base1 == base2:
        alignment_string.append('|')
      else: 
        alignment_string.append(' ')
    print(' '*(seq_2.index(alignedseq2[0]))+'('+''.join(map(str,alignedseq1))+')')
    print(' '*((seq_2.index(alignedseq2[0]))+1)+''.join(alignment_string)+' ') 
    print(seq_2[0:seq_2.index(alignedseq2[0])]+'('+''.join(map(str,alignedseq2))+')'+seq_2[len(seq_2)-(len(seq_2)-(len(alignedseq2)-num+seq_2.index(alignedseq2[0]))):])

   
    


In [3]:
# mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
seq1,seq2,score_matrix,score_loc =runSWalgorithm("/content/drive/MyDrive/Colab Notebooks/cbb752/input.txt", "/content/drive/MyDrive/Colab Notebooks/cbb752/blosum62.txt", -2, -1)
aligned_seq1,aligned_seq2,m= traceback(score_matrix, score_loc,seq1,seq2)
from_where(score_matrix,score_loc[0],score_loc[1])
printout_matrix(seq1,seq2,score_matrix,aligned_seq1,aligned_seq2,m)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


-----------
|Sequences|
-----------
sequence1
SSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSS
sequence2
TFKESSLPSSQPEDSKKTKTTSSNEEEIFTLQVRGKKRFEMLKMINDSLELKDLVPAADQDKYRQKLHSKSTS
--------------
|Score Matrix|
--------------
		S	S	S	P	Q	P	K	K	K	P	L	D	G	E	Y	F	T	L	Q	I	R	G	R	E	R	F	E	M	F	R	E	L	N	E	A	L	E	L	K	D	A	Q	A	G	K	E	P	G	G	S	R	A	H	S	S
	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
T	0	1	1	1	0	0	0	0	0	0	0	0	0	0	0	0	0	5	3	2	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	1
F	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	3	6	4	5	3	2	1	0	0	0	0	6	4	3	6	4	3	2	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
K	0	0	0	0	0	1	0	5	5	5	3	2	1	0	1	1	4	5	3	6	4	4	2	2	1	2	4	7	5	4	8	6	5	4	3	2	1	1	0	5	3	2	1	0	0	5	3	2	1	0	0	2	0	0	0	0
E	0	0	0	0	0	2	0	3	6	6	4	3	4	2	5	3	3	3	2	5	3	4	2	2	7	5	4	9	7	6	6	13	11	10	9	8	7	6	5	4	7	5	4	3	2	3	10	8	7	6	5	4	3	2	1	0
S	0	4	4	4	2	1	1	2	4	6	5	3	3	4	3	3	2	4	2	3	3	2	4	2	5	6	4	7	8	6	5	11	11	12	10	10	8	7	6